In [2]:
from google import genai
from google.genai import types
import time
import json
import os
import cv2
import re
from dotenv import load_dotenv
from PIL import Image
from IPython.display import display
import io

load_dotenv()

# Initialize the client
client = genai.Client()

print("Client initialized!")

Client initialized!


In [21]:
chat_gem3 = client.chats.create(model='gemini-3-pro-preview')

In [29]:
SYSTEM_PROMPT = """
You are a game AI script writer. Your task is to write a Python combat script that defeats the king slime based on video footage you've analyzed.

## GAME STATE

You have access to a `game_state` object with real-time enemy positions:
- `game_state.get_found_entities()` returns a dict of visible enemies: `{"enemy_name": {"x": 800, "y": 400, "found": True}}`. Currently the only thing this will provide you with is the location of the king slime, nothing else not even the small slimes you don't have access to those locations
- The PLAYER is ALWAYS at the CENTER of the screen (x=1280, y=720 on a 2560x1440 display)
- Enemy coordinates are in screen pixels, updated every frame (~60 FPS)

## AVAILABLE ACTIONS
Movement:
- actions.move_left(duration=0.1) - Run left for duration seconds
- actions.move_right(duration=0.1) - Run right for duration seconds  
- actions.move_up(duration=0.1) - Move up for duration seconds
- actions.jump(hold_duration=1.0) - Jump, hold space for duration (default 1s for max height)
- actions.double_jump(first_hold=1.0, second_hold=2.0) - Double jump: hold 1s, release, hold 2s 
- actions.dash_left() - Dash left (double-tap A) - INVINCIBLE during dash
- actions.dash_right() - Dash right (double-tap D) - INVINCIBLE during dash
Combat:
- actions.attack(hold_duration=0.05) - Primary attack (left click)
- actions.attack_at(x, y, hold_duration=0.05) - Aim at pixel position and attack
Utility:
- actions.press_key(key, duration=0.05) - Press a specific key
- actions.hold_key(key, duration=0.5) - Hold a key for duration
- actions.move_mouse(x, y) - Move mouse to absolute position
Tips:
- Keep action durations SHORT (0.05-0.1s) since this runs at 60 FPS
- Dash makes you INVINCIBLE - use it to pass through attacks or deal contact damage
- jump() defaults to 1 second hold for maximum height
- double_jump() is a full 3+ second commitment - use sparingly

## YOUR TASK

Based on the enemy video footage you've seen, write a Python script that defeates the enemy, avoiding being hit by the enemy while also attacking it.

## SCRIPT FORMAT

Your script must have this structure:

```python
\"\"\"
Combat script: [Enemy Name]
Strategy: [Brief description of your approach]

\"\"\"

# Configuration
PLAYER_X = 1280  # Player always at screen center
PLAYER_Y = 720
SAFE_DISTANCE = 400  # Distance to maintain for ranged attacks
DANGER_DISTANCE = 200  # Distance to trigger dodge

def run(game_state, actions):
    '''Called every frame. React to enemy position.'''
    enemies = game_state.get_found_entities()
    
    # Your combat logic here
    # Use the patterns you observed in the video

## IMPORTANT NOTES
- This function runs 60 times per second - keep actions SHORT (0.05-0.1s durations)
- The enemy's X coordinate tells you if they're left (<1280) or right (>1280) of player
- Dash makes you INVINCIBLE - use it to pass through attacks or deal contact damage
- You saw the enemy's attack patterns in the video - use that knowledge to predict and dodge
- Double jump by calling jump() twice with a 0.2s delay between




"""

config = {
    "system_instruction":
          [
            SYSTEM_PROMPT
          ]
}

In [22]:
myfile = client.files.upload(file="fightvideos/0geminiattempt.mp4")
# Wait for processing
while myfile.state.name == "PROCESSING":
    time.sleep(5)
    myfile = client.files.get(name=myfile.name)
# Then use file reference
response = chat_gem3.send_message([
    types.Part(
        file_data=types.FileData(file_uri=myfile.uri),
        video_metadata=types.VideoMetadata(fps=20)
    ),
    """The last script we ran was this:
    \"\"\"
Example behavior script: Dodge when enemy is close.

This script demonstrates how to react to entity positions.
The `run` function is called every frame by script_runner.py.
\"\"\"


# Configuration
DODGE_DISTANCE = 300  # Dodge when enemy is within this many pixels
SCREEN_CENTER_X = 1280  # Half of 2560 resolution


def run(game_state, actions):
    \"\"\"
    Called every frame by script_runner.
    
    Args:
        game_state: GameState object with entity positions
        actions: Actions object with movement/combat helpers
    \"\"\"
    # Get all visible entities
    entities = game_state.get_found_entities()
    
    if not entities:
        return  # Nothing to dodge
    
    # Check each entity
    for name, entity in entities.items():
        x, y = entity["x"], entity["y"]
        
        print(entity.items())
        # Simple dodge logic: if enemy is on left, move right (and vice versa)
        if x < SCREEN_CENTER_X:
            # Enemy on left side, move right
            actions.move_right(0.05)
        else:
            # Enemy on right side, move left
            actions.move_left(0.05)
        
        # Only react to first entity found
        break

## Now write a combat script based on the enemy footage you analyzed. 
    """
], config=config)

print(response.text)

```python
"""
Combat script: King Slime
Strategy: Horizontal kiting. King Slime primarily moves by hopping towards the player and deals contact damage.
The strategy is to maintain a safe distance by constantly running away from the boss's direction.
If the boss closes the gap (Danger Distance), the script uses Dash to utilize invincibility frames and quickly create space.
The player attacks continuously towards the boss's coordinates.
"""

# Configuration
PLAYER_X = 1280  # Player always at screen center
PLAYER_Y = 720
SAFE_DISTANCE = 450  # Distance to maintain to allow reaction time
DANGER_DISTANCE = 250  # Distance to trigger dashes
CRITICAL_DISTANCE = 100 # Distance to trigger panic jumps

def run(game_state, actions):
    '''Called every frame. React to enemy position.'''
    enemies = game_state.get_found_entities()

    # Identify target (King Slime)
    target = None
    for name, entity in enemies.items():
        # Look for the boss, or default to first entity found
        i

In [23]:
myfile = client.files.upload(file="fightvideos/1geminiattempt.mp4")
# Wait for processing
while myfile.state.name == "PROCESSING":
    time.sleep(5)
    myfile = client.files.get(name=myfile.name)
# Then use file reference
response = chat_gem3.send_message([
    types.Part(
        file_data=types.FileData(file_uri=myfile.uri),
        video_metadata=types.VideoMetadata(fps=20)
    ),
    """I ran the last script you provided me with and attached the resultant fight footage.

## Now analyze if you won and if not understand why and write a combat script based on the enemy footage you analyzed. 
    """
], config=config)

print(response.text)

The previous attempt failed because the strategy relied solely on linear kiting (running away). King Slime is faster than the player's walking speed, especially as he takes damage, and his jump arc allows him to land on top of you if you just run in a straight line. The player eventually got cornered or stomped.

To defeat him, we must use the **Dash** ability's invincibility frames to **pass through** the boss when he gets close. This "side-swapping" technique resets the distance and prevents getting cornered.

```python
"""
Combat script: King Slime
Strategy: Aggressive Side-Swapping (The "Passthrough" Method).
Instead of just running away, this script maintains distance until the boss closes the gap.
When King Slime gets too close (within 250px) or is about to land on the player, the script
activates DASH TOWARDS the boss. The dash provides invincibility, allowing the player to 
pass safely through the boss's massive hitbox. Once on the other side, the player runs away 
again to gai

In [24]:
myfile = client.files.upload(file="fightvideos/2geminiattempt.mp4")
# Wait for processing
while myfile.state.name == "PROCESSING":
    time.sleep(5)
    myfile = client.files.get(name=myfile.name)
# Then use file reference
response = chat_gem3.send_message([
    types.Part(
        file_data=types.FileData(file_uri=myfile.uri),
        video_metadata=types.VideoMetadata(fps=20)
    ),
    """I ran the last script you provided me with and attached the resultant fight footage.

## Now analyze if you won and if not understand why and write a combat script based on the enemy footage you analyzed. 
    """
], config=config)

print(response.text)

The previous strategy failed because it attempted to use the dash's invincibility frames to pass through the boss horizontally while he was near the ground. King Slime's hitbox is extremely large, and the dash duration was often insufficient to clear him completely, resulting in the player stopping inside the boss and taking massive contact damage. Additionally, the linear movement eventually cornered the player.

The winning strategy for King Slime involves **"Undercutting"**: waiting for the boss to perform his high jump, and then running *underneath* him to swap sides safely. This resets the arena space without risking a horizontal collision.

```python
"""
Combat script: King Slime
Strategy: Dynamic Kiting with Vertical Undercut.
We mainly kite the boss horizontally to maintain distance. However, to avoid running out of arena space,
we watch for King Slime's high jumps. When he is significantly above the player (Y coordinate check),
we reverse direction and dash/run underneath him.

In [25]:
myfile = client.files.upload(file="fightvideos/3geminiattempt.mp4")
# Wait for processing
while myfile.state.name == "PROCESSING":
    time.sleep(5)
    myfile = client.files.get(name=myfile.name)
# Then use file reference
response = chat_gem3.send_message([
    types.Part(
        file_data=types.FileData(file_uri=myfile.uri),
        video_metadata=types.VideoMetadata(fps=20)
    ),
    """I ran the last script you provided me with and attached the resultant fight footage.

## Now analyze if you won and if not understand why and write a combat script based on the enemy footage you analyzed. 
    """
], config=config)

print(response.text)

The previous strategy failed because the "undercut" maneuver (running under the boss when he jumps) was triggered too loosely or without enough clearance, causing the player to collide with King Slime's massive hitbox. King Slime's jump height varies, and trying to run under a medium-height jump is fatal. Additionally, purely linear kiting leads to running out of arena space.

To win, we need to distinguish between a **High Jump** (safe to run under) and a **Low Hop/Ground Movement** (must run away/jump over). We will also utilize the double jump more effectively to clear him if cornered.

```python
"""
Combat script: King Slime
Strategy: Kite and Switch.
1. Primary Goal: Maintain horizontal distance (Kiting) by running away.
2. Undercut: Only if King Slime is VERY high in the air (High Jump), dash/run underneath him to swap sides and reset arena space.
3. Panic Escape: If he is on the ground and too close, use verticality (Jump/Double Jump) to go OVER him rather than trying to outrun 

In [26]:
myfile = client.files.upload(file="fightvideos/4geminiattempt.mp4")
# Wait for processing
while myfile.state.name == "PROCESSING":
    time.sleep(5)
    myfile = client.files.get(name=myfile.name)
# Then use file reference
response = chat_gem3.send_message([
    types.Part(
        file_data=types.FileData(file_uri=myfile.uri),
        video_metadata=types.VideoMetadata(fps=20)
    ),
    """I ran the last script you provided me with and attached the resultant fight footage.

## Now analyze if you won and if not understand why and write a combat script based on the enemy footage you analyzed. 
    """
], config=config)

print(response.text)

```python
"""
Combat script: King Slime
Strategy: Matador Style Kiting.
The script employs a two-state movement logic based on King Slime's vertical position.
1. Grounded/Low State: When the boss is near the ground, the player kites horizontally AWAY to maintain safety.
2. Airborne State: When the boss performs his signature high jump, the player reverses direction to run UNDERNEATH him.
   This effectively swaps sides, resetting the arena space and preventing the player from being cornered.
   Dash is used to accelerate these movements when proximity is dangerous.
"""

# Configuration
PLAYER_X = 1280
PLAYER_Y = 720
JUMP_THRESHOLD = 550    # Y coordinate to consider the boss "in the air" (Player is at 720)
SAFE_BUFFER = 400       # Distance to maintain when kiting
DANGER_ZONE = 250       # Distance to trigger evasive dashes

def run(game_state, actions):
    '''Called every frame. React to enemy position.'''
    enemies = game_state.get_found_entities()

    target = None
    for name,

In [27]:
myfile = client.files.upload(file="fightvideos/5geminiattempt.mp4")
# Wait for processing
while myfile.state.name == "PROCESSING":
    time.sleep(5)
    myfile = client.files.get(name=myfile.name)
# Then use file reference
response = chat_gem3.send_message([
    types.Part(
        file_data=types.FileData(file_uri=myfile.uri),
        video_metadata=types.VideoMetadata(fps=20)
    ),
    """I ran the last script you provided me with and attached the resultant fight footage.

## Now analyze if you won, and if not understand why and write a combat script based on the enemy footage you analyzed. 
    """
], config=config)

print(response.text)

Based on the analysis of the previous failure, the primary issue was an incorrect trigger for the "Undercut" maneuver. The player attempted to run underneath King Slime when the boss was not sufficiently high in the air, leading to a collision with the boss's massive hitbox. Additionally, the linear kiting eventually led to being cornered.

The new strategy, **"Vertical Gate Kiting"**, sets a stricter threshold for running underneath the boss (only during his distinct high jump) and prioritizes maintaining a safe distance for ranged combat. It also utilizes simple jumping to maintain momentum and clear the small slimes that clutter the ground.

```python
"""
Combat script: King Slime
Strategy: Vertical Gate Kiting.
1. Primary Mode (Ground/Low): When King Slime is on the ground or doing small hops, we run AWAY to maintain distance for ranged attacks. We hop to avoid friction from small slimes.
2. Undercut Mode (High Jump): We only attempt to swap sides when King Slime is at the peak of 

# empress of light attempt

In [3]:
client = genai.Client()

chat_gem3 = client.chats.create(model='gemini-3-pro-preview')

In [31]:
SYSTEM_PROMPT = """
You are a game AI script writer. Your task is to create a plan that defeats the empress of light based on video footage you've analyzed.

## GAME STATE

You have access to a `game_state` object with real-time enemy positions:
- `game_state.get_found_entities()` returns a dict of visible enemies: `{"enemy_name": {"x": 800, "y": 400, "found": True}}`. Currently the only thing this will provide you with is the location of the empress of light, nothing else not even the small slimes you don't have access to those locations
- The PLAYER is ALWAYS at the CENTER of the screen (x=1280, y=720 on a 2560x1440 display)
- Enemy coordinates are in screen pixels, updated every frame (~15 FPS)

## AVAILABLE ACTIONS
Movement:
- you can fly infinitely in any direction 
- you can dash in a given direction (left or right), also if this dash bumps into her you will not take damage
Combat:
- you can shoot her with a gun that is in hand


## YOUR TASK

I need you to figure out what information and game state information you would need to defeat this enemy. Be realistic and clever, things like tracking every single projectile shes throwing is NOT feasible.


"""

config = {
    "system_instruction":
          [
            SYSTEM_PROMPT
          ]
}

In [ ]:

myfile = client.files.upload(file=r"C:\Users\Hhalo\Videos\Dotnet\Dotnet 2026.02.08 - 14.28.52.36.mp4")
# Wait for processing
while myfile.state.name == "PROCESSING":
    time.sleep(5)
    myfile = client.files.get(name=myfile.name)
# Then use file reference
response = chat_gem3.send_message([
    types.Part(
        file_data=types.FileData(file_uri=myfile.uri),
        video_metadata=types.VideoMetadata(fps=20)
    ),
    "I know this seems hard, but be clever and think about what game state information and tools you need for the purpose of writing a python script that uses raw input to evading attack and shoot at her."
], config=config)

print(response.text)

In [ ]:
SYSTEM_PROMPT = """
You are a game AI script writer. Your task is to write a Python combat script that defeats the empress of light based on video footage you've analyzed.

## GAME STATE

You have access to a `game_state` object with real-time enemy positions:
- `game_state.get_found_entities()` returns a dict of visible enemies: `{"enemy_name": {"x": 800, "y": 400, "found": True}}`. Currently the only thing this will provide you with is the location of the empress of light, nothing else
- The PLAYER is ALWAYS at the CENTER of the screen (x=1280, y=720 on a 2560x1440 display)
- Enemy coordinates are in screen pixels, updated every frame (~30 FPS)

## HOW ACTIONS WORK (IMPORTANT)

All actions are **non-blocking intent declarations**. Your `run()` function is called every frame.
You declare what you want to happen THIS frame by calling action methods. After `run()` returns,
the engine applies everything in a single pass (~5-15ms total). This means:

- You can call AS MANY actions as you want per frame with NO performance penalty
- Actions do NOT have duration parameters — they apply for exactly one frame
- To keep moving left, call `actions.move_left()` every frame
- To stop moving, simply stop calling the method — the key is auto-released next frame
- Mouse attack is held while you keep calling `attack_at()` — released when you stop

## AVAILABLE ACTIONS

Movement (call each frame you want the key held):
- actions.move_left()  — Hold left (A key) this frame
- actions.move_right() — Hold right (D key) this frame
- actions.fly_up()     — Hold fly/jump (Space key) this frame
- actions.move_down()  — Hold down (S key) this frame

Dashes (one-shot, queued):
- actions.dash_left()  — Dash left (double-tap A). One dash per frame max
- actions.dash_right() — Dash right (double-tap D). One dash per frame max

Combat:
- actions.attack_at(x, y) — Aim mouse at (x, y) and hold attack this frame

## Your kit
- Your weapon is a gun that needs to be repeatedly clicked to shoot, it has homing bullets that will hit the enemy even if shes off screen, they travel far
- You are very fast and your flight up is also extremely fast

## YOUR TASK

Based on the enemy video footage you've seen, write a Python script that defeats the enemy, avoiding being hit by the enemy while also attacking it.

## SCRIPT FORMAT

Your script must have this structure:

```python
\"\"\"
Combat script: [Enemy Name]
Strategy: [Brief description of your approach]
\"\"\"

# Configuration
PLAYER_X = 1280  # Player always at screen center
PLAYER_Y = 720
SAFE_DISTANCE = 400

def run(game_state, actions):
    '''Called every frame. Declare all actions for this frame.'''
    enemies = game_state.get_found_entities()
    
    # Your combat logic here
    # Call as many actions as needed — they all apply instantly

## Here's an example script:
\"\"\"
Example behavior script: Dodge when enemy is close and attack them constantly.
The `run` function is called every frame by script_runner.py.
\"\"\"

# Configuration
DODGE_DISTANCE = 300
SCREEN_CENTER_X = 1280

def run(game_state, actions):
    entities = game_state.get_found_entities()
    if not entities:
        return

    for name, entity in entities.items():
        x, y = entity["x"], entity["y"]
        
        # Move away from enemy
        if x < SCREEN_CENTER_X:
            actions.move_right()
        else:
            actions.move_left()
        
        # Always shoot at the enemy
        actions.attack_at(x, y)
        break

### Keep in mind 
- All actions are non-blocking. Call multiple actions per frame freely (e.g. move + attack + fly simultaneously)
- Enemy will not always be on your screen when attacking you, so continue to attack their last known location because you can hit them from far away even when theyre not on your screen
- Enemy's name will always be lower case, so "empress" is in the name
- There is an infinite horizontal platform
- Constantly always holding the movement key in the opposite direction of the enemy is highly advised
- dashing constantly is also advised because if you dash into an enemy you won't take damage and enemies sometimes approach you too fast to move out of it so dashing very frequently is advised 
"""

config = {
    "system_instruction":
          [
            SYSTEM_PROMPT
          ]
}

In [5]:

myfile = client.files.upload(file=r"C:\Users\Hhalo\Videos\Dotnet\Dotnet 2026.02.08 - 18.28.45.57.mp4")
# Wait for processing
while myfile.state.name == "PROCESSING":
    time.sleep(5)
    myfile = client.files.get(name=myfile.name)
# Then use file reference
response = chat_gem3.send_message([
    types.Part(
        file_data=types.FileData(file_uri=myfile.uri),
        video_metadata=types.VideoMetadata(fps=20)
    ),
    
    "The script we ran for the latest attached video is the example script in the system prompt, figure out what worked well and what is needed and write the script for your first attempt to defeat this enemy"
], config=config)

print(response.text)

```python
"""
Combat script: Empress of Light
Strategy: High-mobility Run-and-Gun.
1. Kiting: Move horizontally away from the boss at all times to outspace attacks.
2. Speed: Spam dash actions constantly to maintain maximum velocity on the infinite platform.
3. Verticality: Oscillate height (fly/fall) to dodge predictive lance attacks and ground hazards.
4. Offense: Pulse-fire the homing weapon at the last known enemy position.
"""

# Configuration
PLAYER_X = 1280
PLAYER_Y = 720

# State variables
last_enemy_x = 1280  # Default to center
last_enemy_y = 400   # Default to above center
frame_count = 0

def run(game_state, actions):
    global last_enemy_x, last_enemy_y, frame_count
    frame_count += 1
    
    # --- 1. Target Tracking ---
    entities = game_state.get_found_entities()
    target_visible = False
    
    for name, entity in entities.items():
        if "empress" in name.lower():
            last_enemy_x = entity["x"]
            last_enemy_y = entity["y"]
            tar

In [7]:
myfile = client.files.upload(file=r"C:\Users\Hhalo\Videos\Desktop\SUCCESS1empressattempt.mp4")


# Wait for processing
while myfile.state.name == "PROCESSING":
    time.sleep(5)
    myfile = client.files.get(name=myfile.name)
# Then use file reference
response = chat_gem3.send_message([
    types.Part(
        file_data=types.FileData(file_uri=myfile.uri),
        video_metadata=types.VideoMetadata(fps=20)
    ),
    
    "I ran the script you gave me before and attached the newest video, figure out what was the result, what worked well and what is needed and write a new scriptto defeat this enemy"
], config=config)

print(response.text)

Based on the video analysis, the previous kiting strategy was effective at maintaining distance, but the vertical movement was too slow and predictable, causing the bot to fly directly into the "Ethereal Lance" beams and "Prismatic Bolts". Additionally, we need to ensure the gun is clicked properly rather than held, as per the weapon's mechanics.

Here is the improved script. I have tightened the flight pattern to a faster "sawtooth" wave to break enemy tracking, added a buffer to the turning logic to preserve horizontal momentum, and implemented a rapid-click firing cadence.

```python
"""
Combat script: Empress of Light
Strategy: High-speed Runway Kiting with Rapid Vertical Oscillation.
1. Horizontal: Run away from the boss using the infinite platform. Uses a buffer (hysteresis) to prevent loss of speed from turning around too often.
2. Vertical: Faster "Sawtooth" wave pattern. Short bursts of flight followed by falling to disrupt the predictive tracking of the Ethereal Lances.
3. Co

In [ ]:
myfile = client.files.upload(file=r"C:\Users\Hhalo\Videos\PERSONAL ACCOUNT\geminiattempt2 footage.mp4")


# Wait for processing
while myfile.state.name == "PROCESSING":
    time.sleep(5)
    myfile = client.files.get(name=myfile.name)
# Then use file reference
response = chat_gem3.send_message([
    types.Part(
        file_data=types.FileData(file_uri=myfile.uri),
        video_metadata=types.VideoMetadata(fps=20)
    ),
    
    "I ran the script you gave me before and attached the newest video, figure out what was the result, what worked well and what is needed and write a new scriptto defeat this enemy."
], config=config)

print(response.text)

Based on the video analysis, here is the assessment and the revised script.

### Analysis
*   **What Worked:**
    *   **Horizontal Kiting:** The logic to keep the boss to one side and dash away worked well for maintaining distance.
    *   **Combat:** The pulse-fire mechanism successfully operated the weapon.
*   **The Critical Failure:**
    *   **Space Gravity:** The "Sawtooth" pattern failed because the bot drifted upwards over time. Once the character reached the "Space" layer (indicated by the black sky and stars), gravity decreased significantly. The character couldn't fall fast enough to complete the downward part of the wave, got stuck against the world height limit, and became a sitting duck for the boss's attacks.
*   **The Fix:**
    *   **Aggressive Down-Force:** We must explicitly use `actions.move_down()` during the falling phase of the cycle to force the character down against the low gravity.
    *   **Ratio Adjustment:** We will shorten the "Fly Up" duration and lengt